In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
#ee.Authenticate()
#geemap.update_package()

In [3]:
ee.Initialize()

ma_pts = ee.FeatureCollection(
    [ee.Feature(ee.Geometry.Point([-70.842918395996, 42.730953216553]), {'site_id': 'LP HM'}),
     ee.Feature(ee.Geometry.Point([-70.842468261719, 42.731742858887]), {'site_id': 'LP LM'}) ##NOT RIGHT?
    ]
)

In [4]:
##landsat 5 surface reflectance images - MA LTER

maCollectionLS5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2011, 2011,'year')) \
    .filter(ee.Filter.calendarRange(4, 11,'month')) \
    .filterBounds(ma_pts) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(maCollectionLS5.size().getInfo())


10


In [5]:
##landsat 7 surface reflectance images - MA LTER

maCollectionLS7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2012, 2012,'year')) \
    .filter(ee.Filter.calendarRange(4, 11,'month')) \
    .filterBounds(ma_pts) \
    .sort('CLOUD_COVER', True)

vis_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

print(maCollectionLS7.size().getInfo())


10


In [6]:
##landsat 8 surface reflectance images - MA LTER
maCollectionLS8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filter(ee.Filter.calendarRange(2013, 2021,'year')) \
    .filter(ee.Filter.calendarRange(4, 11,'month')) \
    .filterBounds(ma_pts) \
    .sort('CLOUD_COVER', True)

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5} ##be very careful with analysis of l8 vs l5 data - different band numbers!

print(maCollectionLS8.size().getInfo())

114


In [7]:
##Vis params

l5_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B4', 'B3', 'B2'], 
             'gamma': 1.5}

l8_param = {'min': 0, 
             'max': 2000, 
             'bands': ['B5', 'B4', 'B3'], 
             'gamma': 1.5}

In [8]:
maCollectionLS7.aggregate_array('system:id').getInfo()

['LANDSAT/LE07/C01/T1_SR/LE07_012030_20120912',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120624',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120507',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120710',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120827',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20121115',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120405',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120421',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120523',
 'LANDSAT/LE07/C01/T1_SR/LE07_012030_20120811']

In [9]:
##Masking

##Water masking
hansenImage = ee.Image('UMD/hansen/global_forest_change_2015')
datamask = hansenImage.select('datamask')
watermask = datamask.eq(1)

##Cloud/shadow masking
def maskL5sr(image):
  cloudShadowBitMask = 1 << 3
  cloudsBitMask = 1 << 5
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
      .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).divide(10000) \
      .select("B[0-9]*") \
      .copyProperties(image, ["system:time_start"])


In [10]:
##Tidal filtering
def addFLATS(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B6'),
            'RED': image.select('B4'),
            'GREEN': image.select('B3')
        }
    )
    
    return image.addBands(flats.rename('flats'))

##Notes: This is setup for Landsat 8 - apply to l5?

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(-1.57 + 20*(RED-SWIR)/(RED+SWIR) + 68.6*(GREEN-RED)/(GREEN+RED)))', {
            'SWIR': image.select('B5'),
            'RED': image.select('B3'),
            'GREEN': image.select('B2')
        }
    )
    
    return image.addBands(flats.rename('flats'))

In [13]:
##Mapping
##Add Earth Engine dataset
Map = geemap.Map(center=[40,-100], zoom=4)

Map.addLayer(maCollectionLS8.first(), l8_param, "June 1999")

Map.centerObject(ma_pts, 10)
#Map.set_plot_options(add_marker_cluster = True)

##add feature collection points for each plot
Map.addLayer(ma_pts, {}, "plots")

Map

Map(center=[42.73134803794033, -70.84269333029023], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
##IMAGES (masked) - see MA Biomass workbook for sampling dates. +/- 2 weeks sampling date (for the most part)

#1999 - Landsat 5
jun99 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19990528'))).updateMask(watermask)) 
#may, but better cloud cover than 6/15

jul99 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19990715'))).updateMask(watermask))
aug99 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19990816'))).updateMask(watermask))
sep99 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19990901'))).updateMask(watermask))
oct99 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19991019'))).updateMask(watermask))
nov99 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_19991120'))).updateMask(watermask))

#2000
may00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20000514'))).updateMask(watermask))
jun00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20000530'))).updateMask(watermask))
##Within 2 weeks

jul00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20000701'))).updateMask(watermask))
aug00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20000818'))).updateMask(watermask))
sep00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20001005'))).updateMask(watermask))
oct00 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20000919'))).updateMask(watermask))

#2001
may01 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20010501'))).updateMask(watermask))
jun01 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20010618'))).updateMask(watermask))
jul01 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20010704'))).updateMask(watermask))
# aug01 = ee.Image() #NO DATA
sep01 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20010922'))).updateMask(watermask))
oct01 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20011024'))).updateMask(watermask))

#2002
may02 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20020504'))).updateMask(watermask))
jul02 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20020723'))).updateMask(watermask))
sep02 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20020909'))).updateMask(watermask))
nov02 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20021128'))).updateMask(watermask))

#2003
may03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20030507'))).updateMask(watermask))
jun03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20030710'))).updateMask(watermask))
##Within 2 weeks

jul03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20030726'))).updateMask(watermask))
aug03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20030827'))).updateMask(watermask))
sep03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20030912'))).updateMask(watermask))
oct03 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20031014'))).updateMask(watermask))

#2004
# may04 = ee.Image() ##NO DATA
jul04 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20040712'))).updateMask(watermask))
aug04 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20040829'))).updateMask(watermask))
sep04 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20040914'))).updateMask(watermask))
oct04 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20041016'))).updateMask(watermask))

#2005
may05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20050512'))).updateMask(watermask))
jun05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20050613'))).updateMask(watermask))
##Might use 5/28 - clouds

jul05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20050715'))).updateMask(watermask))
aug05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20050816'))).updateMask(watermask))
sep05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20050901'))).updateMask(watermask))
oct05 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20051003'))).updateMask(watermask))

#2006
may06 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20060429'))).updateMask(watermask))
##Within 2 weeks

jun06 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20060616'))).updateMask(watermask))
jul06 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20060718'))).updateMask(watermask))
aug06 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20060803'))).updateMask(watermask))
oct06 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20061006'))).updateMask(watermask))
##Might use 9/20 - clouds

#2007
jun07 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20070619'))).updateMask(watermask))
jul07 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20070705'))).updateMask(watermask))
aug07 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20070822'))).updateMask(watermask))
oct07 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20070923'))).updateMask(watermask))
##Within 2 weeks

#2008
jun08 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20080621'))).updateMask(watermask))
jul08 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20080707'))).updateMask(watermask))
aug08 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20080824'))).updateMask(watermask))
sep08 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20080925'))).updateMask(watermask))
oct08 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20081011'))).updateMask(watermask))

#2009
# may09 = ee.Image() ##NO DATA
jun09 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20090608'))).updateMask(watermask))
jul09 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20090710'))).updateMask(watermask))
aug09 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20090811'))).updateMask(watermask))
##Might use 8/27- clouds

sep09 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20090928'))).updateMask(watermask))
oct09 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20091014'))).updateMask(watermask))

#2010
may10 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20100526'))).updateMask(watermask))
jun10 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20100611'))).updateMask(watermask))
jul10 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20100729'))).updateMask(watermask))
aug10 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20100830'))).updateMask(watermask))
sep10 = addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20100915'))).updateMask(watermask))

#2011
apr11 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20110427'))).updateMask(watermask))
may11 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20110529'))).updateMask(watermask))
##Might use 5/13 - clouds

jul11 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20110716'))).updateMask(watermask))
aug11 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20110817'))).updateMask(watermask))
sep11 =  addFLATSL5(ee.Image(maskL5sr(ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_012030_20110902'))).updateMask(watermask))
# oct11 = ee.Image() ##NO DATA

#2012 - Landsat 7, be careful
may12 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_012030_20120507'))).updateMask(watermask))
jun12 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_012030_20120624'))).updateMask(watermask))
jul12 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_012030_20120710'))).updateMask(watermask))
aug12 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_012030_20120827'))).updateMask(watermask))
sep12 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LE07/C01/T1_SR/LE07_012030_20120912'))).updateMask(watermask))

#2013 - Landsat 8
may13 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20130518'))).updateMask(watermask))
jun13 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20130619'))).updateMask(watermask))
jul13 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20130705'))).updateMask(watermask))
aug13 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20130822'))).updateMask(watermask))
sep13 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20130923'))).updateMask(watermask))

#2014
may14 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20140505'))).updateMask(watermask))
jun14 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20140606'))).updateMask(watermask))
jul14 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20140708'))).updateMask(watermask))
aug14 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20140809'))).updateMask(watermask))
sep14 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20140926'))).updateMask(watermask))

#2015
may15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20150508'))).updateMask(watermask))
jun15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20150524'))).updateMask(watermask))
##Within 2 weeks

jul15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20150711'))).updateMask(watermask))
aug15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20150812'))).updateMask(watermask))
sep15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20150828'))).updateMask(watermask))
##Within 2 weeks

oct15 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20151015'))).updateMask(watermask))

#2016
apr16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160424'))).updateMask(watermask))
may16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160526'))).updateMask(watermask))
jun16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160627'))).updateMask(watermask))
jul16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160713'))).updateMask(watermask))
aug16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160830'))).updateMask(watermask))
sep16 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20160915'))).updateMask(watermask))

#2017
may17 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20170513'))).updateMask(watermask))
jun17 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20170614'))).updateMask(watermask))
jul17 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20170716'))).updateMask(watermask))
aug17 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20170902'))).updateMask(watermask))
##Within several days

oct17 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20171004'))).updateMask(watermask))

#2018
may18 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20180516'))).updateMask(watermask))
jun18 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20180617'))).updateMask(watermask))
jul18 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20180719'))).updateMask(watermask))
aug18 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20180820'))).updateMask(watermask))
oct18 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20181007'))).updateMask(watermask))

#2019
# may19 = ee.Image() ##No data
jun19 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20190604'))).updateMask(watermask))
jul19 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20190807'))).updateMask(watermask))
##Within 2 weeks

aug19 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20190823'))).updateMask(watermask))
sep19 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20190924'))).updateMask(watermask))

#2020
jun20 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20200606'))).updateMask(watermask))
aug20 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20200809'))).updateMask(watermask))

#2021
may21 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20210524'))).updateMask(watermask))
jun21 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20210609'))).updateMask(watermask))
jul21 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20210727'))).updateMask(watermask))
sep21 = addFLATS(ee.Image(maskL5sr(ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_012030_20210913'))).updateMask(watermask))

all_images = ee.ImageCollection.fromImages(
    [
        jun99, jul99, aug99, sep99, oct99, nov99, may00, jun00, jul00, aug00, sep00, oct00, may01, jun01, 
        jul01, sep01, oct01, may02, jul02, sep02, nov02, may03, jun03, jul03, aug03, sep03, oct03, jul04, aug04, 
        sep04, oct04, may05, jun05, jul05, aug05, sep05, oct05, may06, jun06, jul06, aug06, oct06, jun07, jul07,
        aug07, oct07, jun08, jul08, aug08, sep08, oct08, jun09, jul09, aug09, sep09, oct09, may10, jun10, jul10,
        aug10, sep10, apr11, may11, jul11, aug11, sep11, may12, jun12, jul12, aug12, sep12, may13, jun13, jul13, 
        aug13, sep13, may14, jun14, jul14, aug14, sep14, may15, jun15, jul15, aug15, sep15, oct15, apr16, may16, 
        jun16, jul16, aug16, sep16, may17, jun17, jul17, aug17, oct17, may18, jun18, jul18, aug18, oct18, jun19,
        jul19, aug19, sep19, jun20, aug20, may21, jun21, jul21, sep21
    ]
)

In [13]:
Map.addLayer(jun99, {'bands': ['B4',  'B3',  'B2'], 'min': 0, 'max': 0.2}, 'June 1999')
Map.addLayer(aug18, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'August 2018')
Map.addLayer(sep21, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 0.2}, 'September 2021')
Map.addLayer(ma_pts, {}, "plots")

Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [16]:
dem = ee.Image('USGS/3DEP/10m')
##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM', True, 1)

In [24]:
##Exporting pixel values from points

def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ma_pts,
        scale = 30 
    )
    return feature

xtract = all_images.map(addDate).map(rasterExtraction).flatten()

final_extracted = geemap.ee_to_pandas(xtract)

final_extracted['Elevation'] = np.where(final_extracted['site_id'] == 'LP HM', 1.459486, 1.141353) ##see below for elevation

display(final_extracted)

out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'ma_extract.csv')
# final_extracted.to_csv(out_csv, index = False)

##155 rows; would be 228 if there were no cloudy pixels. 25 of the 155 are above flats = 0.1

,site_id,B2,date,B3,B4,B5,B6,B7,flats,B1,Elevation
0,LP HM,0.0675,19990528,0.0728,0.1191,0.1465,0.2902,0.1083,0.000019,0.0420,1.459486
1,LP LM,0.0675,19990528,0.0670,0.1363,0.1513,0.2888,0.1150,0.000119,0.0455,1.141353
2,LP HM,0.0618,19990715,0.0629,0.1668,0.1155,0.2960,0.0712,0.000312,0.0402,1.459486
3,LP LM,0.0695,19990715,0.0629,0.1487,0.1229,0.2951,0.0712,0.009864,0.0439,1.141353
4,LP HM,0.0655,19990901,0.0665,0.1588,0.1126,0.2883,0.0640,0.000718,0.0530,1.459486
...,...,...,...,...,...,...,...,...,...,...,...
150,LP LM,0.0313,20210524,0.0471,0.0460,0.1175,0.0869,0.0486,0.000992,0.0259,1.141353
151,LP HM,0.0258,20210727,0.0591,0.0476,0.1984,0.1291,0.0646,0.032263,0.0144,1.459486
152,LP LM,0.0206,20210727,0.0444,0.0326,0.1514,0.0945,0.0516,0.310532,0.0139,1.141353
153,LP HM,0.0269,20210913,0.0486,0.0449,0.1762,0.1177,0.0594,0.000406,0.0206,1.459486


In [21]:
##Extracting DEM values - only two for the entire study period
def demExtraction(image):
    feature = image.sampleRegions(
        collection = ma_pts,
        scale = 10 
    )
    return feature

dem_xtract = geemap.ee_to_pandas(demExtraction(dem))

display(dem_xtract)

,site_id,elevation
0,LP HM,1.459486
1,LP LM,1.141353


In [ ]:
##Feesh